In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession

In [9]:
sc = SparkContext(master='local', appName='TransformacionesYacciones')

In [11]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

Solo ejecutar `collect()` cuando la cantidad de datos que queremos mostrar es muy pequeña
Ya que cabe la posibilidad de que en nuestros datos existan miles de millones de datos
Y esto es muy **peligroso** en produccion ya que puede tumbar el servidor o puede tar varios minutos
en ejecutarse atrasando toda la produccion

In [12]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=TransformacionesYacciones>

Cargamos un archivo a un RDD es comun ver que utilizen una lambda

In [19]:
path = '/home/armando/sparkCurso/data/'

equipos_olimpicosRDD = sc.textFile(path+'paises.csv') \
    .map(lambda line: line.split(','))

En ligar de `collect()` usaremos `take()` que solo muestra una cantidad determinada de valores

In [24]:
equipos_olimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

La siguiente Query toma los equipos olimpicos toma el parametro de la siglas para al ultimo hacerle metodos de SQL

como lo son el `distinct` que solo toma valores unicos 

y `count` que hace un conteo de todos los resultados unicos

El header es importante que lo toma desde el 0
|id|equipo  |sigla|
|--|--------|-----|
| 0|1       |2    |

In [21]:
equipos_olimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

Esta Query lo que hace es que toma una llave`(x[2])` en este caso es la sigla del pais y un valor`(x[1])`
que es el equipo

- `groupByKey` los agrupara con la llave que es la sigla
- `mapValues(len)` recorera todos los valores y el parametro len retornara cuantos equipos hay en cada pais


In [22]:
equipos_olimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

- `filter` filtra nuestra data de acuerdo a un parametro en este caso obtenemos los equipo de **MEX**

In [33]:
eqipos_MEX = equipos_olimpicosRDD.filter(lambda l: 'MEX' in l)
eqipos_MEX.collect()

[['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['193', 'Chamukina', 'MEX'],
 ['656', 'Mexico', 'MEX'],
 ['657', 'Mexico-1', 'MEX'],
 ['658', 'Mexico-2', 'MEX'],
 ['700', 'Nausikaa 4', 'MEX'],
 ['1053', 'Tlaloc', 'MEX'],
 ['1168', 'Xolotl', 'MEX']]

Para unir data tenemos el metodo de `union` que solo toma como parametro el RDD que queremos indexar

In [25]:
deportista_olimpicoRDD = sc.textFile(path+'deportista.csv', ) \
    .map(lambda l: l.split(','))

deportista_olimpicoRDD2 = sc.textFile(path+'deportista2.csv', ) \
    .map(lambda l: l.split(','))
    
deportista_olimpicoRDD = deportista_olimpicoRDD.union(deportista_olimpicoRDD2)

In [26]:
deportista_olimpicoRDD.count()

135572

Para unir dos RDDs de archivos diferentes tiene que tener una relacion entre ellos una ForeingKey
<image src="data/modelo_relacional.jpg" alt="Descripción de la imagen">

In [29]:
equipos_olimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [30]:
deportista_olimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

Vemos que la FK es el id del equipo 

De igual manera que tomaremos la ultima columna del deportista junto con todos los valores que este tiene menos el ultimo

En el join juntaremos solo el id del pais y las siglas del mismo

`takeSample` toma 3 parametros 
- El primero es que si queremos que los valores se repitan
- El segundo la cantidad de ejemplos que queremos mostrar
- Y el tercero es una semilla que le asignamos

In [53]:
equipo_deportista = equipos_olimpicosRDD.join(deportista_olimpicoRDD).take(6)

In [56]:
deportista_paises = deportista_olimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equipos_olimpicosRDD.map(lambda x: [x[0], x[2]]))

deportista_paises.take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

Obtenemos el archivo de resultado

In [38]:
resultadoRDD = sc.textFile(path+'resultados.csv', ) \
    .map(lambda l: l.split(','))

Filtramos omitiendo a todos aquellos que no han ganado una medalla

In [39]:
resultado_ganadorRDD = resultadoRDD.filter(lambda l: 'NA' not in l[1])

In [40]:
resultado_ganadorRDD.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [54]:
deportista_paises.join(resultado_ganadorRDD).take(6)

[('1090',
  ((['9150', 'Yelena Yuryevna Bechke Petrova Ellis ', '2', '26', '158', '48'],
    'EUN'),
   'Bronze')),
 ('1090',
  ((['9819', 'Gennady Vladimirovich Belyakov', '1', '23', '171', '84'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['41896', 'Andrey Vladimirovich Gorokhov', '1', '23', '185', '92'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['64345', 'Sergey Valeryevich Kruglov', '1', '31', '0', '0'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['70906', 'Igor Vladimirovich Lobanov', '1', '22', '181', '78'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['90775', 'Irina Vladimirovna Palina', '2', '22', '162', '60'], 'EUN'),
   'Bronze'))]